In [1]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 15.4 MB/s 
     |████████████████████████████████| 486 kB 77.3 MB/s 
     |████████████████████████████████| 13.4 MB 81.1 MB/s 
     |████████████████████████████████| 880 kB 62.3 MB/s 
     |████████████████████████████████| 3.3 MB 39.3 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=fff7b9bc908a8c462e96f4b11a78629ac44f1ff65aac268c348df6cf054de929
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c063047260c665dab4c6f98cd59a95801c8c9510616259e223ebdf49c3f6a526
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built ipadic sacremoses


In [2]:
import numpy as np
import pandas as pd
import string
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, BertJapaneseTokenizer, BertModel
from torch import cuda
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from transformers import logging


In [3]:
batch_size = 16
max_len = 512

In [4]:
df_train = pd.read_csv("./drive/MyDrive/Colab_Notebooks/data/IMDb/IMDb_train.tsv", sep="\t", header=None)
df_train = df_train.iloc[:, 0:2]
df_train.columns = ["text", "label"]
print(df_train.shape)
df_train.head()

(25000, 2)


,text,label
0,The Unborn is a pretty good low-budget horror ...,1
1,Vincente Minnelli directed some of the most ce...,1
2,"The first time I saw this, I didn't laugh too ...",1
3,This is a great movie for all Generation X'ers...,1
4,I first saw this absolutely riveting documenta...,1


In [5]:
df_test = pd.read_csv("./drive/MyDrive/Colab_Notebooks/data/IMDb/IMDb_test.tsv", sep="\t", header=None)
df_test = df_test.iloc[:, 0:2]
df_test.columns = ["text", "label"]
print(df_test.shape)
df_test.head()

(25000, 2)


,text,label
0,"Susan Sarandon is, for lack of a better word, ...",1
1,Seeing Laurel without Hardy in a film seems st...,1
2,I was recently at a sleepover birthday party w...,1
3,This movie took me by surprise. The opening cr...,1
4,A widely unknown strange little western with m...,1


In [6]:
df_train["label"].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [7]:
df_train["text"].map(len).median()

979.0

In [8]:
df_train["text"].map(len)

0         639
1        4018
2         921
3        1582
4        1526
         ... 
24995     472
24996     751
24997     964
24998    6041
24999    1405
Name: text, Length: 25000, dtype: int64

# 前処理

In [9]:
def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text


In [10]:
df_train["text"].map(preprocessing_text)
preprocessing_text(df_train["text"][0])

'The Unborn is a pretty good low budget horror movie exploiting the fears associated with pregnancy .  It s very well acted by the always good Brooke Adams and b movie stalwart James Karen ,  although the supporting cast is pretty average for a b grader .  The music ,  by Gary Numan of all people ,  is good too .  Henry Dominic s script is quite intelligent for this sort of thing ,  although there is a hint of misogyny about it .  Rodman Fender s direction is merely adequate ,  and there are some unnecessary cheap scares .  If you re a fan of Adams ,  whose movie career is nowhere near as illustrious as it should be ,  check it out  she s great ,  as always . '

# データセット作成

In [11]:
class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer, max_len):
    self.X = X
    self.y = y
    # self.uid = uid
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.y)

  def encode(self, tokenizer, text):

      # 前処理
      text = preprocessing_text(text)
      

      inputs = tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          padding = 'max_length',
          truncation = True
      )
      return inputs

  def __getitem__(self, index):
    text = self.X[index]
    label = self.y[index]
    # userID = self.uid[index]
    ids = []
    mask = []
    inputs = self.encode(tokenizer=self.tokenizer, text=text)
    ids.append(torch.LongTensor(inputs['input_ids']))
    mask.append(torch.LongTensor(inputs['attention_mask']))

    return {
      'ids': ids,
      'mask': mask,
      'label': label,
      'text':text,
      # 'userID':userID
    }

In [12]:
# tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [13]:
X = df_train["text"].values
y = df_train["label"].values

In [14]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, train_size=0.75)

print(len(X_train))
print(len(y_train))
print(len(X_eval))
print(len(y_eval))

X_test = df_test["text"].values
y_test = df_test["label"].values
print(len(X_test))
print(len(y_test))

18750
18750
6250
6250
25000
25000


In [15]:
dataset_train = CreateDataset(X_train, y_train, tokenizer, max_len=max_len)
dataset_eval = CreateDataset(X_eval, y_eval, tokenizer, max_len=max_len)
dataset_test = CreateDataset(X_test, y_test, tokenizer, max_len=max_len)

print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

18750
6250
25000


In [16]:
dataset_train[1]

{'ids': [tensor([  101,  2160,   117,  5397,  1618,  1190,  1139, 10761,  1104,  4735,
          12872,   119, 10188,  1199,  1104,  1103,  6209,  1127,  2785,  1560,
           1105,  1103,  1301,  1874,  1108, 11858,   119,  1135,  1108,  1912,
           1104,  1176, 20337, 12128,  5636,  1103,  5377,  4373,  8855,   119,
            139, 16830,   122,   119,  1332,  1103,  4067,  1344,  1278,  1137,
           2134,  1132, 10751,  5367,  5558,  1107,  1103,  3119,   117,  1917,
            156,  2328,  1162,  1867,  1110,  1593,  1126,  6129, 15882,  1121,
          26942,  1820,   119,  1109,  1645,  1164,  1103,  1992,  7209,  1174,
           2636,  3576,   119,   123,   119,  3982,  3036, 24819,  1942,   170,
          10909, 27412,  9577,  1121, 26942,   124,   119,  4981,  1103,  1864,
           1115,  1122,  1261,  1282,  1107,  1357,   117,  1184,  1103,  2630,
           1225,  1103,  2523,  1138,  1106,  1202,  1114,  2687, 10390,  1179,
          12004, 14924,  1191,  1

# データローダ作成

In [17]:
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, pin_memory=True)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, shuffle=False, pin_memory=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, pin_memory=True)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dataloader_train, "val": dataloader_eval}

In [18]:
tmp = next(iter(dataloader_train))
print(tmp["ids"][0].size())
print(tmp["label"])
tmp["ids"][0][0]


torch.Size([16, 512])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1])


tensor([  101,  1109, 14760, 12948,  1108,  1189,  1113,  1103,  1159,  1165,
         1593,  1451,  9545,  1125,  1122,   188, 15604, 21155,  1186,   119,
         1284,  1125,  6340,  2021,  3099,   117, 13342,  1441,   117,  3318,
         5075,   188,  1105,  1115,  1108,  1198,  1111, 12302,  1116,   119,
         1109, 14760, 12948,  1338, 28117,  1643, 25443,  1193,  1523,  1272,
         1280,  1106, 10552, 12948,  1110,  1932,  1451,  1399,   188, 12178,
          119,  1109,  4928,  1110,  6061,   119,  3198,  4044, 10552, 12948,
         4157, 11907,  4935,  4793,  2491,  3264,  1297,  1107,  1117,  1632,
         1653,  1402,   117,  1119,  1144,  2712,  8781,  1676,  1105,  1632,
         1282,  1106,  1250,  1112,   170, 10552, 12948,   119,  4753,  8435,
         1132,  1909,  1106,  1117, 11385,  1107,  1103,  1532,  1104, 13392,
          146,  8900,  2564, 12008,  9019, 24682,   188,  4008,  9326,  1424,
          117,  7320,  4528, 23722,  9374,  1117,  1676,  1105, 

# BERTモデル

In [19]:
# model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", output_attentions=True, output_hidden_states=True)
model = BertModel.from_pretrained("bert-base-cased", output_attentions=True, output_hidden_states=True)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [39]:
from torch import nn


class BertForLivedoor(nn.Module):
    '''BERTモデルにPosiNegaの2クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにクラス予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は2クラス
        self.cls = nn.Linear(in_features=768, out_features=2)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    # def forward(self, input_ids):
    #     '''
    #     input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
    #     '''

    #     # BERTの基本モデル部分の順伝搬
    #     # 順伝搬させる
    #     result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

    #     # sequence_outputの先頭の単語ベクトルを抜き出す
    #     vec_0 = result[0]  # 最初の0がsequence_outputを示す
    #     vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語(cls)の全768要素
    #     vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
    #     output = self.cls(vec_0)  # 全結合層

    #     return output

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        token_type_ids： [batch_size, sequence_length]の、各単語が1文目なのか、2文目なのかを示すid
        attention_mask：Transformerのマスクと同じ働きのマスキングです
        output_all_encoded_layers：最終出力に12段のTransformerの全部をリストで返すか、最後だけかを指定
        attention_show_flg：Self-Attentionの重みを返すかのフラグ
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        if attention_show_flg == True:
            '''attention_showのときは、attention_probsもリターンする'''
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        elif attention_show_flg == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)

        # 入力文章の1単語目[CLS]の特徴量を使用して、ポジ・ネガを分類します
        vec_0 = encoded_layers[:, 0, :]
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_sizeに変換
        out = self.cls(vec_0)

        # attention_showのときは、attention_probs（1番最後の）もリターンする
        if attention_show_flg:
            return out, attention_probs
        elif not attention_show_flg:
            return out

In [40]:
# モデル構築
net = BertForLivedoor()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')

ネットワーク設定完了


In [41]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True

In [42]:
# 最適化手法の設定
import torch.optim as optim


# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

In [45]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch["ids"][0].to(device)  # 文章
                labels = batch["label"].to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs, token_type_ids=None, attention_mask=None,
                                  output_all_encoded_layers=False, attention_show_flg=False)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net

# 学習・評価

In [46]:
# 学習・検証を実行する。
num_epochs = 1
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------


AttributeError: ignored

In [28]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

# モデル評価用データ
labels_all = []
preds_all = []

for batch in tqdm(dataloader_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    inputs = batch["ids"][0].to(device)  # 文章
    labels = batch["label"].to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs)


        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

        # f1計算用
        labels_all.extend(batch["label"].to('cpu').detach().numpy())
        preds_all.extend(preds.to('cpu').detach().numpy())

# 正解率
epoch_acc = epoch_corrects.double() / len(dataloader_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dataloader_test.dataset), epoch_acc))

  2%|▏         | 28/1563 [00:15<14:14,  1.80it/s]


KeyboardInterrupt: ignored

In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print(f"正解率: {accuracy_score(labels_all, preds_all):.3f}")
print(f"適合率: {precision_score(labels_all, preds_all):.3f}")
print(f"再現率: {recall_score(labels_all, preds_all):.3f}")
print(f"F1: {f1_score(labels_all, preds_all):.3f}")

ValueError: ignored

# Attentionの可視化

In [37]:
# BertForIMDbで処理

# ミニバッチの用意
batch = next(iter(dataloader_test))

# GPUが使えるならGPUにデータを送る
inputs = batch["ids"][0].to(device)  # 文章
labels = batch["label"].to(device)  # ラベル

net_trained.eval()
outputs, attention_probs = net_trained(
    inputs, 
    token_type_ids=None, 
    attention_mask=None,
    output_all_encoded_layers=False, 
    attention_show_flg=True
    )

_, preds = torch.max(outputs, 1)  # ラベルを予測


TypeError: ignored

In [35]:
inputs.size()

torch.Size([16, 512])